# Leetcode Section 5

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Question 1: Market Analysis 1

`Users`

| user_id | join_date  | favorite_brand |
|---------|------------|----------------|
| 1       | 2018-01-01 | Lenovo         |
| 2       | 2018-02-09 | Samsung        |
| 3       | 2018-01-19 | LG             |
| 4       | 2018-05-21 | HP             |

`Orders`

| order_id | order_date | item_id | buyer_id | seller_id |
|----------|------------|---------|----------|-----------|
| 1        | 2019-08-01 | 4       | 1        | 2         |
| 2        | 2018-08-02 | 2       | 1        | 3         |
| 3        | 2019-08-03 | 3       | 2        | 3         |
| 4        | 2018-08-04 | 1       | 4        | 2         |
| 5        | 2018-08-04 | 1       | 3        | 4         |
| 6        | 2019-08-05 | 2       | 2        | 4         |

`Items`

| item_id | item_brand |
|---------|------------|
| 1       | Samsung    |
| 2       | Lenovo     |
| 3       | LG         |
| 4       | HP         |


Write an SQL query to find for each user, the join date and the number of orders they made as a buyer in 2019.

| buyer_id  | join_date  | orders_in_2019 |
|-----------|------------|----------------|
| 1         | 2018-01-01 | 1              |
| 2         | 2018-02-09 | 2              |
| 3         | 2018-01-19 | 0              |
| 4         | 2018-05-21 | 0              |


In [88]:
# data preparation
Users = pd.DataFrame({"user_id":[1,2,3,4],
             "join_date":pd.to_datetime(["2018-01-01","2018-02-09","2018-01-19","2018-05-21"]),
             "favorite_brand":["Lenovo","Samsung","LG","HP"]})
Orders = pd.DataFrame({"order_id":np.arange(1,7),
                      "order_date":pd.to_datetime(["2019-08-01","2018-08-02","2019-08-03","2018-08-04","2018-08-04","2019-08-05"]),
                      "item_id":[4,2,3,1,1,2],
                      "buyer_id":[1,1,2,4,3,2],
                      "seller_id":[2,3,3,2,4,4]})
Items = pd.DataFrame({"item_id":np.arange(1,5),
                     "item_brand":["Samsung","Lenovo","LG","HP"]})

To compute the number of orders each user places, we need to group the data by user_id

In [86]:
# first we filter out data of 2019
order_2019 = Orders[Orders.order_date.map(lambda x:x.year) == 2019]

Consider that not each user places order and that we need to query the order info of each user, we need to use outer join so that we can pint the info of each user.

In [101]:
merged_df = pd.merge(Users,order_2019,left_on="user_id",right_on="buyer_id",how='left')
# to stop the groupby function ignoring null values, we need to first fill them with certain value
res = merged_df_nona.groupby("user_id").agg({"join_date":"min","order_id":"size"})
res.iloc[2,1]=0
res.iloc[3,1]=0
res

,join_date,order_id
user_id,,
1,2018-01-01,1
2,2018-02-09,2
3,2018-01-19,0
4,2018-05-21,0


## Question 2: Market Analysis II

`users`

| Column Name    | Type    |
|----------------|---------|
| user_id        | int     |
| join_date      | date    |
| favorite_brand | varchar |

user_id is the primary key of this table.
This table has the info of the users of an online shopping website where users can sell and buy items.

`orders`

| Column Name   | Type    |
|---------------|---------|
| order_id      | int     |
| order_date    | date    |
| item_id       | int     |
| buyer_id      | int     |
| seller_id     | int     |

order_id is the primary key of this table.
item_id is a foreign key to the Items table.
buyer_id and seller_id are foreign keys to the Users table.

`items`

| Column Name   | Type    |
|---------------|---------|
| item_id       | int     |
| item_brand    | varchar |

item_id is the primary key of this table.


Write an SQL query to find for each user, whether the brand of the second item (by date) they sold is their favorite brand. If a user sold less than two items, report the answer for that user as no.

It is guaranteed that no seller sold more than one item on a day.

The query result format is in the following example:

`res`

| seller_id | 2nd_item_fav_brand |
|-----------|--------------------|
| 1         | no                 |
| 2         | yes                |
| 3         | yes                |
| 4         | no                 |



In [102]:
Orders = pd.DataFrame({"order_id":np.arange(1,7),
                      "order_date":pd.to_datetime(["2019-08-01","2019-08-02","2019-08-03","2019-08-04","2019-08-04","2019-08-05"]),
                      "item_id":[4,2,3,1,1,2],
                      "buyer_id":[1,1,2,4,3,2],
                      "seller_id":[2,3,3,2,4,4]})

In [112]:
### first we merge the itmes table with orders table so we can look up the name of brand for each seller
order_item_df = pd.merge(Orders,Items,on="item_id")
order_item_df["sell_rank"] = order_item_df["order_date"].groupby(order_item_df["seller_id"]).rank(method="first",ascending=True)

# then we need to outer join the table with user table
brand_df = pd.merge(Users,order_item_df,left_on="user_id",right_on="seller_id",how='left')
boolean_condi = brand_df.sell_rank.map(lambda x:np.isnan(x))
rank_2 = brand_df.sell_rank==2
brand_df_ = brand_df[boolean_condi | rank_2]
brand_df_["2nd_item_fav_brand"] = np.where(brand_df_.favorite_brand==brand_df_.item_brand,"yes","no")
res = brand_df_[["user_id","2nd_item_fav_brand"]].rename(columns={"user_id":"seller_id"})
res


/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,seller_id,2nd_item_fav_brand
0,1,no
2,2,yes
4,3,yes
5,4,no


## Question 3: Product Price at a Given Date

Write an SQL query to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

`Products`

| product_id | new_price | change_date |
|------------|-----------|-------------|
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |

(product_id, change_date) is the primary key of this table.

`res`

| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |



In [113]:
# data preparartion
Products = pd.DataFrame({"product_id":[1,2,1,1,2,3],
             "new_price":[20,50,30,35,65,20],
             "change_date":pd.to_datetime(["2019-08-14","2019-08-14","2019-08-15","2019-08-16","2019-08-17","2019-08-18"])})
Products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


**We can split the data into two parts: before 2019-08-16 and after 2019-08-16**

- for data with date smaller than or equal to 2019-08-16, we just need to find their maximum change_date and corresponding price
- for data with date greater than 2019-08-16, the price of 2019-08-16 must be 10

In [135]:
products = Products.copy()

# first we filter out data with date before 2019-08-16
products_816 = products[products.change_date<=datetime(2019,8,16)]
products_816["rank"] = products_816["change_date"].groupby(products_816["product_id"]).rank(method='first',ascending=False)
before_aug16 = products_816[products_816["rank"]==1]
prods = pd.DataFrame({"product_id":products.product_id.unique()})
merged_prods = pd.merge(prods,before_aug16,on="product_id",how="left")
merged_prods["new_price"] = merged_prods["new_price"].fillna(10)
res = merged_prods[["product_id","new_price"]].rename(columns={"new_pirce":"price"})
res

/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,product_id,new_price
0,1,35.0
1,2,50.0
2,3,10.0


## Question 4: Tournament Winners

The winner in each group is the player who scored the maximum total points within the group. In the case of a tie, the lowest player_id wins.

Write an SQL query to find the winner in each group.

The query result format is in the following example:

`Players`

| player_id | group_id   |
|-----------|------------|
| 15        | 1          |
| 25        | 1          |
| 30        | 1          |
| 45        | 1          |
| 10        | 2          |
| 35        | 2          |
| 50        | 2          |
| 20        | 3          |
| 40        | 3          |

player_id is the primary key of this table.

`Matches`

| match_id   | first_player | second_player | first_score | second_score |
|------------|--------------|---------------|-------------|--------------|
| 1          | 15           | 45            | 3           | 0            |
| 2          | 30           | 25            | 1           | 2            |
| 3          | 30           | 15            | 2           | 0            |
| 4          | 40           | 20            | 5           | 2            |
| 5          | 35           | 50            | 1           | 1            |

- match_id is the primary key of this table.
- Each row is a record of a match, first_player and second_player contain the player_id of each match.
- first_score and second_score contain the number of points of the first_player and second_player respectively.
- You may assume that, in each match, players belongs to the same group.

`res`

| group_id  | player_id  |
|-----------|------------|
| 1         | 15         |
| 2         | 35         |
| 3         | 40         |


In [136]:
# data preparation
Players = pd.DataFrame({"player_id":[15,25,30,45,10,35,50,20,40],
             "group_id":[1,1,1,1,2,2,2,3,3]})
Matches = pd.DataFrame({"match_id":[1,2,3,4,5],
                       "first_player":[15,30,30,40,35],
                       "second_player":[45,25,15,20,50],
                       "first_score":[3,1,2,5,1],
                       "second_score":[0,2,0,2,1]})

The winner in each group is the player who scored the maximum total points within the group. In the case of a tie, the lowest player_id wins. This informaiton of question tells us several things:

- We need to compute the the maximum score within each group. To get the maximum number of score, we can `rank` the score of score
- In case of tie, the lowest score player_id wins, meaning that we may have to group the team id and select the smaller player_id


The relationship between `player` and `matches` needs to be thoroughly examined. Users can be either first_player or second_player. So to compute the total score of each player within each team, we need to concatenate them vertically.

In [153]:
# first we find the group id of first player
first_players = pd.merge(Players,Matches,left_on="player_id",right_on="first_player",how='inner')
second_players = pd.merge(Players,Matches,left_on="player_id",right_on="second_player",how='inner')

# then concatenate the two tables
full_players = pd.concat([first_players,second_players])

# then group the data by group id and user_id to compute the total score of each player within each team
full_players['score'] = np.where(full_players.player_id==full_players["first_player"],full_players.first_score,full_players.second_score)
full_players["total_score"] = full_players['score'].groupby(full_players["player_id"]).transform('sum')
full_players

,player_id,group_id,match_id,first_player,second_player,first_score,second_score,score,total_score
0,15,1,1,15,45,3,0,3,3
1,30,1,2,30,25,1,2,1,3
2,30,1,3,30,15,2,0,2,3
3,35,2,5,35,50,1,1,1,1
4,40,3,4,40,20,5,2,5,5
0,15,1,3,30,15,2,0,0,3
1,25,1,2,30,25,1,2,2,2
2,45,1,1,15,45,3,0,0,0
3,50,2,5,35,50,1,1,1,1
4,20,3,4,40,20,5,2,2,2


We have computed the total score of each player. Now we need to rank them within each group using `rank` function.

In [166]:
full_players['rank'] = full_players["total_score"].groupby(full_players["group_id"]).rank(method="dense",ascending=False)
rank_1 = full_players["rank"] == 1
rank_one_players = full_players[rank_1]

# then we need to find players with smaller id in the case of tie

res = rank_one_players.groupby(["group_id","rank"],as_index=False).agg({"player_id":"min"})[["group_id","player_id"]]
res

,group_id,player_id
0,1,15
1,2,35
2,3,40


# Q5: League Statistics

Write an SQL query to report the statistics of the league. The statistics should be built using the played matches where the winning team gets three points and the losing team gets no points. If a match ends with a draw, both teams get one point.

Each row of the result table should contain:

- `team_name` - The name of the team in the Teams table.
- `matches_played` - The number of matches played as either a home or away team.
- `points` - The total points the team has so far.
- `goal_for` - The total number of goals scored by the team across all matches.
- `goal_against` - The total number of goals scored by opponent teams against this team across all matches.
- `goal_diff` - The result of goal_for - goal_against.

Return the result table in descending order by points. If two or more teams have the same points, order them in descending order by goal_diff. If there is still a tie, order them by team_name in lexicographical order.

The query result format is in the following example:

`Teams`

| team_id | team_name |
|---------|-----------|
| 1       | Ajax      |
| 4       | Dortmund  |
| 6       | Arsenal   |


`Matches`

| home_team_id | away_team_id | home_team_goals | away_team_goals |
|--------------|--------------|-----------------|-----------------|
| 1            | 4            | 0               | 1               |
| 1            | 6            | 3               | 3               |
| 4            | 1            | 5               | 2               |
| 6            | 1            | 0               | 0               |


`Result`

| team_name | matches_played | points | goal_for | goal_against | goal_diff |
|-----------|----------------|--------|----------|--------------|-----------|
| Dortmund  | 2              | 6      | 6        | 2            | 4         |
| Arsenal   | 2              | 2      | 3        | 3            | 0         |
| Ajax      | 4              | 2      | 5        | 9            | -4        |


In [32]:
Teams = pd.DataFrame({"team_id":[1,4,6],"team_name":["Ajax","Dortmund","Arsenal"]})
Matches = pd.DataFrame({'home_team_id':[1,1,4,6],
                      "away_team_id":[4,6,1,1],
                      "home_team_goals":[0,3,5,0],
                      "away_team_goals":[1,3,2,0]})

In [49]:
teams = Teams.copy()
matches= Matches.copy()
# first query all matches records of home teams and away teams
home_team_df = pd.merge(teams,matches,left_on="team_id",right_on="home_team_id")
away_team_df = pd.merge(teams,matches,left_on="team_id",right_on="away_team_id")
# concatenate the two table by rows to obtain the matches records of all teams
teams_df = pd.concat([home_team_df,away_team_df],axis=0,ignore_index=True)
teams_df

,team_id,team_name,home_team_id,away_team_id,home_team_goals,away_team_goals
0,1,Ajax,1,4,0,1
1,1,Ajax,1,6,3,3
2,4,Dortmund,4,1,5,2
3,6,Arsenal,6,1,0,0
4,1,Ajax,4,1,5,2
5,1,Ajax,6,1,0,0
6,4,Dortmund,1,4,0,1
7,6,Arsenal,1,6,3,3


Compute the statistics based on the table

- `matches_played` can be calculated by grouping `team_id` and counting the number of records
- `points` can be computed by writing functions to separately compute the score of team goals

In [64]:
# then iterate the dataframe by rows

def compute_points(data):
    """compute the total points of a team has scored 
        so far across all matches """
    if data.loc["team_id"] == data.loc["home_team_id"]:
        if data.loc["home_team_goals"] < data.loc["away_team_goals"]:
            return 0
        elif data.loc["home_team_goals"] == data.loc["away_team_goals"]:
            return 1
        else:
            return 3
    else:
        if data.loc["home_team_goals"] < data.loc["away_team_goals"]:
            return 3
        elif data.loc["home_team_goals"] == data.loc["away_team_goals"]:
            return 1
        else:
            return 0

def goal_for(data):
    """the total number of goals scored by the team across all matches"""
    
    if data.loc["team_id"] == data.loc["home_team_id"]:
        return data.loc['home_team_goals']
    elif data.loc["team_id"] == data.loc["away_team_id"]:
        return data.loc["away_team_goals"]
    
def goal_against(data):
    """the total number of goals scored by opponent teams across all matches"""
    if data.loc["team_id"] == data.loc["home_team_id"]:
        return data.loc['away_team_goals']
    elif data.loc["team_id"] == data.loc["away_team_id"]:
        return data.loc["home_team_goals"]

In [71]:
# first compute the number of matches
res_df = teams_df.groupby("team_name",as_index=False).agg({"team_id":"count"}).rename(columns={"team_id":"matches_played"})
res_df

,team_name,matches_played
0,Ajax,4
1,Arsenal,2
2,Dortmund,2


In [78]:
# total points
res_df["points"] = teams_df.groupby("team_id").apply(lambda group:group.apply(lambda row:compute_points(row),axis=1)).\
groupby("team_id").sum().values
# goal for
res_df["goal_for"] = teams_df.groupby("team_id").apply(lambda group:group.apply(lambda row:goal_for(row),axis=1)).\
groupby("team_id").sum().values
# goal against
res_df["goal_against"] = teams_df.groupby("team_id").apply(lambda group:group.apply(lambda row:goal_against(row),axis=1)).\
groupby("team_id").sum().values
res_df["goal_diff"] =  res_df["goal_for"] - res_df["goal_against"]
# sort the datframe based on the question
res = res_df.sort_values(by=["points","goal_diff","team_name"],ascending=[False,False,True])
res


,team_name,matches_played,points,goal_for,goal_against,goal_diff
1,Arsenal,2,6,6,2,4
2,Dortmund,2,2,3,3,0
0,Ajax,4,2,5,9,-4


# Q6: Find Interview Candidates


Write an SQL query to report the name and the mail of all interview candidates. A user is an interview candidate if at least one of these two conditions is true:

- The user won any medal in three or more consecutive contests.
- The user won the gold medal in three or more different contests (not necessarily consecutive).

Return the result table in any order.

The query result format is in the following example:

`Contests`

| contest_id | gold_medal | silver_medal | bronze_medal |
|------------|------------|--------------|--------------|
| 190        | 1          | 5            | 2            |
| 191        | 2          | 3            | 5            |
| 192        | 5          | 2            | 3            |
| 193        | 1          | 3            | 5            |
| 194        | 4          | 5            | 2            |
| 195        | 4          | 2            | 1            |
| 196        | 1          | 5            | 2            |

contest_id is the primary key for this table.

This table contains the LeetCode contest ID and the user IDs of the gold, silver, and bronze medalists.

It is guaranteed that any consecutive contests have consecutive IDs and that no ID is skipped.


`Users`

| user_id | mail               | name  |
|---------|--------------------|-------|
| 1       | sarah@leetcode.com | Sarah |
| 2       | bob@leetcode.com   | Bob   |
| 3       | alice@leetcode.com | Alice |
| 4       | hercy@leetcode.com | Hercy |
| 5       | quarz@leetcode.com | Quarz |

user_id is the primary key for this table.


`Result`

| name  | mail               |
|-------|--------------------|
| Sarah | sarah@leetcode.com |
| Bob   | bob@leetcode.com   |
| Alice | alice@leetcode.com |
| Quarz | quarz@leetcode.com |




In [89]:
# data preparation
Contests = pd.DataFrame({'contest_id':np.arange(190,197),
             "gold_medal":[1,2,5,1,4,4,1],
             "sliver_medal":[5,3,2,3,5,2,5],
             "bronze_medal":[2,5,3,5,2,1,2]})
Users = pd.DataFrame({"user_id":np.arange(1,6),
                     "mail":["sarah@leetcode.com","bob@leetcode.com","alice@leetcode.com","hercy@leetcode.com","quarz@leetcode.com"],
                     "name":["Sarah","Bob","Alice","Hercy","Quarz"]})

- Find out players who win three or more gold medals

In [110]:
condidate_df1 = Contests.groupby("gold_medal",as_index=False).agg({"contest_id":"count"},).rename(columns={"contest_id":"gold_cnt"})
mask = condidate_df1["gold_cnt"]>=3
candidate_df1 = condidate_df1[mask].rename(columns={"gold_medal":'user_id'}).drop("gold_cnt",axis=1)
candidate_df1

,user_id
0,1


- Find out candidates who win any medal in three or more consecutive contests

In [112]:
# unpivot the dataframe, transforming the df from wide format to long format
melted_df = pd.melt(Contests,id_vars="contest_id",value_vars=["gold_medal","sliver_medal","bronze_medal"],var_name="medal",value_name="winner")
melted_df["row_number"] = melted_df["contest_id"].groupby(melted_df["winner"]).rank(method="first")
melted_df["flag"] = melted_df["contest_id"] - melted_df["row_number"]
temp = melted_df.groupby(["winner","flag"],as_index=False).agg({'contest_id':"count"}).rename(columns={"contest_id":"winning_contests"})
winning_users = temp["winner"][temp["winning_contests"]>=3].unique()
candidate_df2 = pd.DataFrame({"user_id":winning_users})

- Concatenate two dataframes

In [117]:
res = pd.concat([candidate_df1,candidate_df2]).merge(Users,on="user_id")[["name","mail"]]
res

,name,mail
0,Sarah,sarah@leetcode.com
1,Bob,bob@leetcode.com
2,Alice,alice@leetcode.com
3,Quarz,quarz@leetcode.com


# Q7: Suspicious Bank Accounts

`Accounts`

| account_id | max_income |
|------------|------------|
| 3          | 21000      |
| 4          | 10400      |

account_id is the primary key for this table.
Each row contains information about the maximum monthly income for one bank account.


`Transactions`

| transaction_id | account_id | type     | amount | day                 |
|----------------|------------|----------|--------|---------------------|
| 2              | 3          | Creditor | 107100 | 2021-06-02 11:38:14 |
| 4              | 4          | Creditor | 10400  | 2021-06-20 12:39:18 |
| 11             | 4          | Debtor   | 58800  | 2021-07-23 12:41:55 |
| 1              | 4          | Creditor | 49300  | 2021-05-03 16:11:04 |
| 15             | 3          | Debtor   | 75500  | 2021-05-23 14:40:20 |
| 10             | 3          | Creditor | 102100 | 2021-06-15 10:37:16 |
| 14             | 4          | Creditor | 56300  | 2021-07-21 12:12:25 |
| 19             | 4          | Debtor   | 101100 | 2021-05-09 15:21:49 |
| 8              | 3          | Creditor | 64900  | 2021-07-26 15:09:56 |
| 7              | 3          | Creditor | 90900  | 2021-06-14 11:23:07 |

- transaction_id is the primary key for this table.
- Each row contains information about one transaction.
- type is ENUM ('Creditor','Debtor') where 'Creditor' means the user deposited money into their account and 'Debtor' means the user withdrew money from their account.
- amount is the amount of money depositied/withdrawn during the transaction.


Write an SQL query to report the IDs of all suspicious bank accounts.

A bank account is suspicious **if the total income exceeds the max_income for this account for two or more consecutive months. The total income of an account in some month is the sum of all its deposits in that month (i.e., transactions of the type 'Creditor').**

Return the result table in ascending order by transaction_id.

The query result format is in the following example:


In [118]:
# data preparation
Accounts = pd.DataFrame({"account_id":[3,4],
                        "max_income":[21000,10400]})
Transactions = pd.DataFrame({"transaction_id":[2,4,11,1,15,10,14,19,8,7],
             "account_id":[3,4,4,4,3,3,4,4,3,3],
             "type":["Creditor","Creditor","Debtor","Creditor","Debtor","Creditor","Creditor","Debtor","Creditor","Creditor"],
             "amount":[107100,10400,58800,49300,75500,102100,56300,101100,64900,90900],
             "day":pd.to_datetime(["2021-06-02 11:38:14","2021-06-20 12:39:18","2021-07-23 12:41:55","2021-05-03 16:11:04","2021-05-23 14:40:20",
                   "2021-06-15 10:37:16","2021-07-21 12:12:25","2021-05-09 15:21:49","2021-07-26 15:09:56","2021-06-14 11:23:07"])})

In [131]:
# find out accounts with income greater than the max income

# add one variable to indicate year and month
accounts = Accounts.copy()
transactions = Transactions.copy()
transactions["year_month"] = transactions["day"].apply(lambda x:datetime.strftime(x,"%Y-%m"))

# compute the total income per month for each user
temp = transactions.groupby(["year_month","account_id"],as_index=False).agg({"amount":"sum"})
temp = pd.merge(temp,accounts,on="account_id")
mask = temp.amount > temp.max_income
SusUsers_df = temp[mask]
SusUsers_df

,year_month,account_id,amount,max_income
0,2021-05,3,75500,21000
1,2021-06,3,300100,21000
2,2021-07,3,64900,21000
3,2021-05,4,150400,10400
5,2021-07,4,115100,10400


In [176]:
# then filter out users with total income greater than the max income for three or more consecutive months
from pandas.tseries.offsets import MonthBegin
from dateutil.relativedelta import relativedelta

SusUsers_df["row_number"] = SusUsers_df["year_month"].groupby(SusUsers_df["account_id"]).rank(method="first",ascending=True).\
apply(lambda x:relativedelta(months=x))
SusUsers_df["flag"] = SusUsers_df["year_month"].apply(lambda x:datetime.strptime(x,"%Y-%m")) - SusUsers_df.row_number
temp = SusUsers_df.groupby(["account_id","flag"],as_index=False).agg({"amount":"count"})
mask = temp.amount>=3
res = temp[mask][["account_id"]]
res

/var/folders/qq/f4fnr8tn703_g56r17sjvj5r0000gn/T/ipykernel_1225/1569116220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SusUsers_df["row_number"] = SusUsers_df["year_month"].groupby(SusUsers_df["account_id"]).rank(method="first",ascending=True).\
/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:1187: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(
/var/folders/qq/f4fnr8tn703_g56r17sjvj5r0000gn/T/ipykernel_1225/1569116220.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,account_id
0,3
